In [7]:
import random
from lang import load_data_int_seq
from models.rl.env import IntegerSequenceEnv
from models.rl.agents.agent_mcts.main import train_mcts

%load_ext autoreload
%autoreload 2

MAX_PENALTY_MAGNITUDE = 999.0

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
output_lang, input_lang, train, X_test, y_test = load_data_int_seq()
train = train[:1]

In [9]:
env = IntegerSequenceEnv({"data": train, "output_length": 9, "input_lang": input_lang, "output_lang": output_lang})

In [12]:
env.reset()

[-1, -1, -1, -1, -1, -1, -1, -1, -1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [11]:
train_mcts(env)

ValueError: too many values to unpack (expected 4)